In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
import string

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def mask_pii(model, tokenizer, text, aggregate_redaction=True):

    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Get the model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted labels
    predictions = torch.argmax(outputs.logits, dim=-1)

    # Convert token predictions to word predictions
    encoded_inputs = tokenizer.encode_plus(text, return_offsets_mapping=True, add_special_tokens=True, max_length=512, truncation=True)
    offset_mapping = encoded_inputs['offset_mapping']

    masked_text = list(text)
    is_redacting = False
    redaction_start = 0
    current_pii_type = ''

    for i, (start, end) in enumerate(offset_mapping):
        if start == end:  # Special token
            continue

        label = predictions[0][i].item()
        if label != model.config.label2id['O']:  # Non-O label
            pii_type = model.config.id2label[label]
            pii_type = pii_type.split('-')[-1]  # Remove B- or I- prefix
            if not is_redacting:
                is_redacting = True
                redaction_start = start
                current_pii_type = pii_type
            elif not aggregate_redaction and pii_type != current_pii_type:
                # End current redaction and start a new one
                apply_redaction(masked_text, redaction_start, start, current_pii_type, aggregate_redaction)
                redaction_start = start
                current_pii_type = pii_type
        else:
            if is_redacting:
                apply_redaction(masked_text, redaction_start, end, current_pii_type, aggregate_redaction)
                is_redacting = False

    # Handle case where PII is at the end of the text
    if is_redacting:
        apply_redaction(masked_text, redaction_start, len(masked_text), current_pii_type, aggregate_redaction)

    return ''.join(masked_text)
    
def apply_redaction(masked_text, start, end, pii_type, aggregate_redaction):
    # Remove content in the range to prepare for redaction insertion
    for j in range(start, end):
        masked_text[j] = ''

    # Set replacement text based on redaction preference
    if aggregate_redaction:
        replacement = '[redacted]'
    else:
        replacement = f'[{pii_type}]'

    # Ensure a space before the replacement if it’s not the start of the text and the previous character isn’t a space
    if start > 0 and masked_text[start - 1] not in (' ', ''):
        replacement = ' ' + replacement

    # Ensure a space after the replacement if the following character isn’t punctuation or a space
    if end < len(masked_text) and masked_text[end] not in string.punctuation + " ":
        replacement = replacement + ' '

    # Place the replacement at the start index
    masked_text[start] = replacement

In [2]:
# Example from https://huggingface.co/iiiorg/piiranha-v1-detect-personal-information/discussions/4
example_text = "My name is Obee Nobi and I live at 432423 Deka St., Tanooti. My phone number is 5455 123 4567."

In [3]:
# Piiranha model
model_name = "iiiorg/piiranha-v1-detect-personal-information"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
model.to(device)

print("Detailed redaction:")
masked_example_detailed = mask_pii(model, tokenizer, example_text, aggregate_redaction=False)
print(masked_example_detailed)

Detailed redaction:
My name is [GIVENNAME][SURNAME] I live at [BUILDINGNUM][STREET]., [CITY] My phone number is 5455 123 4567.


In [4]:
# Our Model 1
model_name = "/teamspace/studios/this_studio/submission/results/checkpoint-19533"
tokenizer = AutoTokenizer.from_pretrained(model_name)
modelv1 = AutoModelForTokenClassification.from_pretrained(model_name)
modelv1.to(device)
print("Detailed redaction:")
masked_example_detailed = mask_pii(modelv1, tokenizer, example_text, aggregate_redaction=False)
print(masked_example_detailed)

Detailed redaction:
My name is [GIVENNAME] [SURNAME] I live at 432 [ZIPCODE] ka St., [CITY] My phone number is [TELEPHONENUM]


In [5]:
# Our Model 2 
model_name = "/teamspace/studios/this_studio/submission/result_v2/checkpoint-16839"
tokenizer = AutoTokenizer.from_pretrained(model_name)
modelv1 = AutoModelForTokenClassification.from_pretrained(model_name)
modelv1.to(device)
print("Detailed redaction:")
masked_example_detailed = mask_pii(modelv1, tokenizer, example_text, aggregate_redaction=False)
print(masked_example_detailed)

Detailed redaction:
My name is [GIVENNAME][SURNAME] I live at [BUILDINGNUM][STREET], [CITY] phone number is [TELEPHONENUM]


In [6]:
# Our Model 3 (experimental)
model_name = "results-noise-5/checkpoint-28065"
tokenizer = AutoTokenizer.from_pretrained(model_name)
modelv1 = AutoModelForTokenClassification.from_pretrained(model_name)
modelv1.to(device)
print("Detailed redaction:")
masked_example_detailed = mask_pii(modelv1, tokenizer, example_text, aggregate_redaction=False)
print(masked_example_detailed)

Detailed redaction:
My name is [GIVENNAME][SURNAME] I live at [BUILDINGNUM] ka St., [CITY] phone number is [TELEPHONENUM]
